# Day 14: Extended Polymerization
https://adventofcode.com/2021/day/14

### Part 1

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

"This polymer grows quickly...."  That is ominous.  Regardless, we will go for the simple solution and see what part 2 brings.

In [1]:
from collections import Counter
from functools import cache
from typing import Tuple
from advent_of_code import day_14
from advent_of_code import utils

%load_ext blackcellmagic

In [2]:
def read_input(input_file: str) -> Tuple[str, dict]:
    """
    Parse input file into polymer template name (str)
    and rules (dict) with the form k:v->element pair:element
    """
    polymer: str = None
    rules: dict = {}  # k:v -> element pair:element

    with open(input_file) as f:
        for line in f:
            if line.rstrip() and not "->" in line:
                polymer = line.rstrip()

            elif line.rstrip() and "->" in line:
                pair, insertion = line.rstrip().split(" -> ")
                rules[pair] = insertion

    return polymer, rules

def expand_pair(e1: str, e2: str, n: int, rules: dict) -> str:
    """Recurisve expansion of the pair down to n-steps. Return expanded pair (<str>)."""
    insertion = rules.get(e1 + e2, "")
    if n == 1:
        return e1 + insertion + e2
    else:
        insertion = rules.get(e1 + e2, "")
        return expand_pair(e1, insertion, n - 1, rules)[:-1] + expand_pair(
            insertion, e2, n - 1, rules
        )

def expand(starting_polymer: str, rules: dict, steps: int = 1) -> str:
    """applies rules to starting polymer, and returns output."""

    resulting_polymer: str = ""

    for e1, e2 in zip(starting_polymer, starting_polymer[1:]):
        resulting_polymer = resulting_polymer + expand_pair(e1, e2, steps, rules)[:-1]

    resulting_polymer = resulting_polymer + starting_polymer[-1]
    return resulting_polymer

def occurrance_count_check_counter(c: Counter) -> int:
    """ """
    return c.most_common()[0][1] - c.most_common()[-1][1]

def occurrance_count_check(polymer: str) -> int:
    """
    Returns the quantity of the most common element minus the
    least common element.
    """
    c = Counter(polymer)
    return occurrance_count_check_counter(c)

In [3]:
# Apply test on page.
test_input_file = utils.test_input_location(day=14)
polymer, rules = read_input(test_input_file)

assert expand(polymer, rules, 1) == "NCNBCHB"
assert expand(polymer, rules, 2) == "NBCCNBBBCBHCB"
assert expand(polymer, rules, 3) == "NBBBCNCCNBBNBNBBCHBHHBCHB"
assert expand(polymer, rules, 4) == "NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB"
assert occurrance_count_check(expand(polymer, rules, 10)) == 1588

In [4]:
input_file = utils.input_location(day=14)
polymer, rules = read_input(input_file)
occurrance_count_check(expand(polymer, rules, 10))

2549

### Part 2
Apply 40 Steps.  ... Part 1 likely won't scale ... :( 

As assumed, this runs too long.
Even attempting to cache the results to ```expand_pair``` does not yield enough speed up.

```python
test_input_file = utils.test_input_location(day=14)
polymer, rules = read_input(test_input_file)
assert occurrance_count_check(step(polymer, 40)) == 2188189693529
```


In [5]:
@cache
def pair_count(pair: str, step: int, rules: day_14.FrozenDict) -> Counter:
    """
    A recursive function to track pair expansions in a Counter object.
    Returns counter
    """
    if step == 0 or pair not in rules:
        return Counter()
    else:
        c = Counter(rules.get(pair))
        c.update(pair_count(pair[0] + rules.get(pair), step - 1, rules))
        c.update(pair_count(rules.get(pair) + pair[1], step - 1, rules))
        return c

def expand_counts(starting_polymer: str, rules: dict, steps: int = 1) -> Counter:
    """
    Wrapper around pair_count that calls for each starting pair combination
    Returns counter object.
    """
    # instead of a global variable, we use a hashable dictionary (custom class)
    # this will enable us to benefit from the cache logic above.
    frozen_rules = day_14.FrozenDict(rules)
    c = Counter(starting_polymer)
    for e1, e2 in zip(starting_polymer, starting_polymer[1:]):
        c.update(pair_count(e1 + e2, steps, frozen_rules))

    return c

In [6]:
# replicate tests from part 1
test_input_file = utils.test_input_location(day=14)
polymer, rules = read_input(test_input_file)
assert expand_counts(polymer, rules, 1)  == Counter("NCNBCHB")
assert expand_counts(polymer, rules, 2)  == Counter("NBCCNBBBCBHCB")
assert expand_counts(polymer, rules, 3)  == Counter("NBBBCNCCNBBNBNBBCHBHHBCHB")
assert expand_counts(polymer, rules, 4)  == Counter("NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB")
assert occurrance_count_check_counter(expand_counts(polymer, rules, 10)) == 1588

In [7]:
# Replicate part 1
input_file = utils.input_location(day=14)
polymer, rules = read_input(input_file)
occurrance_count_check_counter(expand_counts(polymer, rules, 10))

2549

In [8]:
input_file = utils.input_location(day=14)
polymer, rules = read_input(input_file)
occurrance_count_check_counter(expand_counts(polymer, rules, 40))

2516901104210